In [19]:
# | hide

from IPython.display import Markdown as md
from IPython.display import Code


# Cryptocurrency analysis with FastStream

In this tutorial we will generate two FastStream applications.

The first application will retrieve current prices for several cryptocurrencies from the web and publish messages to a Kafka topic.

The second application will then retrieve messages with information about cryptocurrencies from the topic and calculate the average price of each cryptocurrency in the last few seconds.

## Fetch and publish app

Create directory for your application, create and enter the virtual environment and install `faststream-gen`:

```sh
# create and enter app firectory
mkdir fetch_and_publish_app
cd fetch_and_publish_app

# create and enter venv
python -m venv venv_app1
source venv_app1/bin/activate

# install faststream-gen
pip install --upgrade pip
pip install faststream-gen
```

Now let's create an application that retrieves information about cryptocurrencies from the web and publishes messages to a Kafka topic.

The most important step is to create a text description of the application we want to generate. Inside it is necessary to define where information about cryptocurrencies can be found and in what format they will be, the scheme of messages that we will produce in the Kafka topic.

`description_fetch_publish.txt`:

In [20]:
# | hide

with open('../../docs_src/tutorial/scrape_and_publish_description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [21]:
# | echo: false

md(description)


```text
Create faststream application which will fetch Bitcoins current price and publish it to new_data topic. 
Application should fetch the data every 2 seconds.

Message which will be produced is JSON with the two attributes:
    - price: non negative float (it represents current pice of Bitcoin)
    - currency: string (it represents the currency of the price, e.g USD, EUR...)

Curent price of Bitcoin can be fetched by simple GET request to 'https://api.coinbase.com/v2/prices/BTC-USD/spot'
Curent price of Ethereum can be fetched by simple GET request to 'https://api.coinbase.com/v2/prices/ETH-USD/spot'

Use different keys Bitcoin and Ethereum when puclishing:
    - For Bitcoin, encode string 'BTC' and use it as key.
    - For Ethereum, encode string 'ETH' and use it as key.
    
The response of this GET request is a JSON and you can get information about the currency in response['data']['base']
and the information about the price in response['data']['amount']
 
```


To create described `faststream-gen` application, copy the previous description and paste it inside the `description_fetch_publish.txt` file.

And run:

```sh
faststream_gen -i description_fetch_publish.txt
```

This command will generate `github` repository with `app/application.py` and `tests/test_application.py` inside.

`app/application.py`:

In [22]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/scrape_and_publish_app.py', language='python')
md(f"```python\n{code}\n```")

```python
from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

import requests
import asyncio

class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., description="Current price of the cryptocurrency")
    currency: str = Field(..., description="Currency of the price")

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

new_data_publisher = broker.publisher("new_data")

async def fetch_and_publish_crypto_price(
    url: str,
    key: str,
    logger: Logger,
    context: ContextRepo,
    time_interval: int = 2,
) -> None:
    while context.get("app_is_running"):
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            price = data["data"]["amount"]
            currency = data["data"]["base"]
            crypto_price = CryptoPrice(price=price, currency=currency)
            await new_data_publisher.publish(crypto_price, key=key.encode("utf-8"))
        else:
            logger.warning(f"Failed to fetch data from {url}")
        await asyncio.sleep(time_interval)


@app.on_startup
async def app_setup(context: ContextRepo):
    context.set_global("app_is_running", True)


@app.on_shutdown
async def app_shutdown(context: ContextRepo):
    context.set_global("app_is_running", False)


@app.after_startup
async def publish_crypto_prices(logger: Logger, context: ContextRepo):
    logger.info("Starting publishing:")

    bitcoin_url = "https://api.coinbase.com/v2/prices/BTC-USD/spot"
    ethereum_url = "https://api.coinbase.com/v2/prices/ETH-USD/spot"

    publish_tasks = [
        asyncio.create_task(
            fetch_and_publish_crypto_price(bitcoin_url, "BTC", logger, context)
        ),
        asyncio.create_task(
            fetch_and_publish_crypto_price(ethereum_url, "ETH", logger, context)
        ),
    ]
    context.set_global("publish_tasks", publish_tasks)

```

## Calculate the moving average app

Now let's create the second application. Open **new terminal**, create directory for your application, create and enter the virtual environment and install `faststream-gen`:

```sh
# create and enter app firectory
mkdir calculate_moving_average_app
cd calculate_moving_average_app

# create and enter venv
python -m venv venv_app2
source venv_app2/bin/activate

# install faststream-gen
pip install --upgrade pip
pip install faststream-gen
```

This application shoud calculate the price mean of the last 3 messages received at the 'new_data' topic for each cryptocurrency and puublish the price mean to the price_mean topic.

`description_calculate_mean.txt`:

In [15]:
# | hide

with open('../../docs_src/tutorial/calculate_mean_description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [16]:
# | echo: false

md(description)


```text
Create faststream application for consuming messages from the new_data topic. 
This topic needs to use partition key.

new_data messages use JSON with two attributes:
    - price: non negative float (it represents current pice of the crypto)
    - currency: string (it represents the currency of the price, e.g BTC, ETH...)

Application should save each message to a dictionary (global variable) - partition key should be usded as a dictionary key and value should be a List of prices.
Calculate the price mean of the last 3 messages for the given partition key
Publish the price mean to the price_mean topic and use the same partition key which the new_data topic is using.
 
```


To create described `faststream-gen` application, copy the previous description and paste it inside the `description_calculate_mean.txt` file.

And run:
```sh
faststream_gen -i description_calculate_mean.txt
```

This command will generate `github` repository with `app/application.py` and `tests/test_application.py` inside.

`app/application.py`:

In [18]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/calculate_mean_app.py', language='python')
md(f"```python\n{code}\n```")

```python
from typing import Dict, List

from pydantic import BaseModel, Field

from faststream import Context, ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

class NewData(BaseModel):
    price: float = Field(
        ..., examples=[1000], description="Current price of the crypto"
    )
    currency: str = Field(
        ..., examples=["BTC"], description="Currency of the price"
    )

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

to_price_mean = broker.publisher("price_mean")

@app.on_startup
async def app_setup(context: ContextRepo):
    message_history: Dict[str, List[float]] = {}
    context.set_global("message_history", message_history)

@broker.subscriber("new_data")
async def on_new_data(
    msg: NewData,
    logger: Logger,
    context: ContextRepo,
    key: bytes = Context("message.raw_message.key"),
) -> None:
    logger.info(f"{msg=}")

    message_history = context.get("message_history")

    partition_key = key.decode("utf-8")
    if partition_key not in message_history:
        message_history[partition_key] = []

    message_history[partition_key].append(msg.price)
    context.set_global("message_history", message_history)

    if len(message_history) >=3:
        price_mean = sum(message_history[partition_key][-3:]) / 3
        await to_price_mean.publish(price_mean, key=key)

```

## Start localhost Kafka broker


`faststream-gen` also generated `scripts` directory.
```sh
# make all shell scripts executable
chmod +x scripts/*.sh
# start local kafka broker
./scripts/start_kafka_broker_locally.sh
```

```console
[+] Running 2/2
 ⠿ Network scripts_default  Created                                                                                                             0.1s
 ⠿ Container bitnami_kafka  Started 
```

## Run the applications

### Fetch and publish app

To start application for fetching and publishing cryptocurrency data, run inside the `fetch_and_publish_app` directory the following command:
```sh
faststream run  app.application:app
```
```console
2023-09-15 13:41:21,948 INFO     - FastStream app starting...
2023-09-15 13:41:22,144 INFO     -      |            - Starting publishing:
2023-09-15 13:41:22,144 INFO     - FastStream app started successfully! To exit press CTRL+C
```

### Calculate the moving average app

To start application for calculating the moving average, run inside the `calculate_moving_average_app` directory the following command:
```sh
faststream run  app.application:app
```
```console
2023-09-15 13:56:47,245 INFO     - FastStream app starting...
2023-09-15 13:56:47,428 INFO     - new_data |            - `OnNewData` waiting for messages
2023-09-15 13:56:47,621 INFO     - FastStream app started successfully! To exit press CTRL+C
2023-09-15 13:56:48,314 INFO     - new_data | 13675-1694 - Received
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - msg=NewData(price=1624.235, currency='ETH')
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - Processed
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Received
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - msg=NewData(price=26485.545, currency='BTC')
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Processed
2023-09-15 13:56:50,491 INFO     - new_data | 13677-1694 - Received
2023-09-15 13:56:50,492 INFO     - new_data | 13677-1694 - msg=NewData(price=1624.235, currency='ETH')
...
```

You can see in the terminal the that the application is reading the messages from the `price_mean` topic.

### Subscribe directly to local kafka broker topic

To check if the `calculate_moving_average_app` is publishing messages to the `price_mean` topic, run the following comands:

 ```sh
# make all shell scripts executable
chmod +x scripts/*.sh
# subscribe to local kafka broker and read from the price_mean topic
./scripts/subscribe_to_kafka_broker_locally.sh price_mean
```
```console
BTC     26405.745
ETH     1621.3733333333332
BTC     26404.865
ETH     1621.375
BTC     26404.865
...
```